# Spotify Signup

### Introduction

In this lesson we'll work with the spotify web API.  As we'll see, uses a different authentication flow than we have previously seen.

### Signing Up with Spotify

To start, go to the [Spotify Developer Login](https://developer.spotify.com/dashboard/login) and login with your Spotify account, or create a new one.

<img src="https://github.com/eng-6-22/github-apis/blob/master/4-spotify-web-api/spotify-login.png?raw=1" width="60%">

Once logged in, click on Create a New App, and then signup for non-commercial use.

Afterwards, you will be directed to the client id and secret pages.

### Authenticating with the API

By now you should have your client id and client secret codes.

To see how to use them, read through the [client credentials flow on Spotify](https://developer.spotify.com/documentation/general/guides/authorization-guide/#client-credentials-flow).  Don't worry, we'll wait.

> It explains that before making a request, we need to use the `client_id` and `client_secret` to retreive an auth token.

The documentation does not explicitly show us how to get an auth token using Python.  But lucky for us, this [this stackoverflow post](https://stackoverflow.com/questions/30557409/spotify-api-post-call-response-415) does.

Copy the code below and confirm that you can return an access token with the code.

In [1]:
from urllib.parse import urlencode

import requests

from config import SPOTIFY_CLIENT_ID as CLIENT_ID
from config import SPOTIFY_CLIENT_SECRET as CLIENT_SECRET

In [2]:
TOKEN_URL ='https://accounts.spotify.com/api/token'
DATA = {'grant_type': 'client_credentials'}

BASE_URL = 'https://api.spotify.com'
SEARCH_ENDPOINT = '/v1/search'

After confirming that you can retrieve an access token, turn the code into a function called `get_access_token` that takes in arguments of `client_id` and `client_secret` and returns the access token. 

In [3]:
def get_access_token():
    r = requests.post(TOKEN_URL, data=DATA, auth=(CLIENT_ID, CLIENT_SECRET))
    return r.json().get('access_token')

In [4]:
TOKEN = get_access_token()  # Your access token will be different

HEADERS = {"Authorization": f"Bearer {TOKEN}"}

### Making a search

Now that we have an access token, it's time for us to use the Spotify API to get some information about music.  The main component that we would like to use in the API is the [discover page](https://developer.spotify.com/discover/#search).

We can start by making a request for information about an musical artist.  Look at the [search documents for an artist](https://developer.spotify.com/documentation/web-api/reference/search/search/) to see how.

Write a function called `artist_search_url` that takes in an `artist_name`, and returns the corresponding correct url to search for the artist. 

In [5]:
def artist_search_url(artist_name):
    params = {'q': artist_name, 'type': 'artist'}
    return f'{BASE_URL}{SEARCH_ENDPOINT}?{urlencode(params)}'

In [6]:
paul_simon_request_url = artist_search_url('paul simon')
paul_simon_request_url
# 'https://api.spotify.com/v1/search?q=paul simon&type=artist'

'https://api.spotify.com/v1/search?q=paul+simon&type=artist'

Write a function called `make_request` that takes in the token, and url and makes the request.  You can reference [this stackoverflow post](https://stackoverflow.com/questions/29931671/making-an-api-call-in-python-with-an-api-that-requires-a-bearer-token) to see how to provide the access token into headers.  Just use a `get` request instead of a post request.

In [7]:
def make_request(url):
    return requests.get(url, headers=HEADERS).json()

You can check that you write the functin correctly with the following by checking the return value of the function below.

In [8]:
paul_simon_data = make_request(paul_simon_request_url)

In [9]:
paul_simon_data.get('artists').get('items')[0]

{'external_urls': {'spotify': 'https://open.spotify.com/artist/2CvCyf1gEVhI0mX6aFXmVI'},
 'followers': {'href': None, 'total': 1884367},
 'genres': ['classic rock',
  'folk',
  'folk rock',
  'mellow gold',
  'permanent wave',
  'rock',
  'roots rock',
  'singer-songwriter',
  'soft rock'],
 'href': 'https://api.spotify.com/v1/artists/2CvCyf1gEVhI0mX6aFXmVI',
 'id': '2CvCyf1gEVhI0mX6aFXmVI',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb1f764c8f69b595efe77e1c45',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab676161000051741f764c8f69b595efe77e1c45',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f1781f764c8f69b595efe77e1c45',
   'width': 160}],
 'name': 'Paul Simon',
 'popularity': 67,
 'type': 'artist',
 'uri': 'spotify:artist:2CvCyf1gEVhI0mX6aFXmVI'}

In [10]:
# paul_simon_data

# {'artists': {'href': 'https://api.spotify.com/v1/search?query=paul+simon&type=artist&offset=0&limit=20',
#   'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2CvCyf1gEVhI0mX6aFXmVI'},
#     'followers': {'href': None, 'total': 1219295},
#     'genres': ['classic rock',
#      'folk',
#      'folk rock',
#      'mellow gold',
#      'permanent wave',
#      'rock',
#      'roots rock'

> When you see that it's correct, feel free to comment out the output.

Next write a method that extracts the `name`, `id` and popularity from the json response for an artist.

In [11]:
def extract_artist_info(search_resp):
    items = search_resp.get('artists').get('items')
    return [{'name': d['name'], 'id': d['id'], 'popularity': d['popularity']} for d in items]

In [12]:
paul_simon_info = extract_artist_info(paul_simon_data)
paul_simon_info[:5]
# {'name': 'Paul Simon', 'id': '2CvCyf1gEVhI0mX6aFXmVI', 'popularity': 74}

[{'name': 'Paul Simon', 'id': '2CvCyf1gEVhI0mX6aFXmVI', 'popularity': 67},
 {'name': 'Paul Simonon', 'id': '62bYKAZ5EdmG5Aca9dtVan', 'popularity': 49},
 {'name': 'Simon & Garfunkel',
  'id': '70cRZdQywnSFp9pnc2WTCE',
  'popularity': 69},
 {'name': 'Paul Simon', 'id': '7saY0ZRzZgJJJgRw4yn7Vo', 'popularity': 4},
 {'name': 'Roberto Alagna', 'id': '2AyySZZK01JT6ZSYZ2CWgf', 'popularity': 41}]

In [13]:
real_paul_simon = paul_simon_info[0]

real_paul_simon

{'name': 'Paul Simon', 'id': '2CvCyf1gEVhI0mX6aFXmVI', 'popularity': 67}

## Digging Deeper

Now that we have successfully requested and extracted information from the Spotify API, it's time to go deeper.  

Let's use the artist's `id`, to search for more information about the artist.

* Finding the `top_tracks`

With the Spotify API, we can find the top tracks of an artist.  Write a method that takes in an `artist_id`, and returns url to request an artist's top tracks. 

In [14]:
def top_tracks_url(artist_id):
    return f'https://api.spotify.com/v1/artists/{artist_id}/top-tracks?market=US'

In [15]:
simon_id = real_paul_simon['id']

simon_artist_url = top_tracks_url(simon_id)
simon_artist_url
# 'https://api.spotify.com/v1/artists/2CvCyf1gEVhI0mX6aFXmVI/top-tracks'

'https://api.spotify.com/v1/artists/2CvCyf1gEVhI0mX6aFXmVI/top-tracks?market=US'

Now use the url to make the request to the API.

In [16]:
simon_tracks = make_request(simon_artist_url)

In [17]:
# simon_tracks

# {'tracks': [{'album': {'album_type': 'album',
#     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2CvCyf1gEVhI0mX6aFXmVI'},
#       'href': 'https://api.spotify.com/v1/artists/2CvCyf1gEVhI0mX6aFXmVI',
#       'id': '2CvCyf1gEVhI0mX6aFXmVI',
#       'name': 'Paul Simon',
#       'type': 'artist',
#       'uri': 'spotify:artist:2CvCyf1gEVhI0mX6aFXmVI'}],

Write a method called `extract_track_info` that takes in a single track from the response, and returns a smaller dictionary of specified keys.  We'll provide a default argument of `keys = ['name', 'id', 'popularity]`, but if provided a list of different keys, it should extract them. 

> Look up dictionary comprehension to see how to do this.

In [18]:
tracks = simon_tracks['tracks']

first_track = tracks[0]

first_track.keys()

# dict_keys(['album', 'artists', 'disc_number', 'duration_ms',
# 'explicit', 'external_ids', 'external_urls', 'href', 'id',
# 'is_local', 'is_playable', 'name', 'popularity', 
# 'preview_url', 'track_number', 'type', 'uri'])

dict_keys(['album', 'artists', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'is_playable', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [19]:
def extract_track_info(track_response, keys = ['name', 'id', 'popularity']):
    return {k: track_response[k] for k in keys}

In [20]:
extract_track_info(first_track)

{'name': 'You Can Call Me Al',
 'id': '0qxYx4F3vm1AOnfux6dDxP',
 'popularity': 76}

In [21]:
extract_track_info(first_track, keys = ['name', 'popularity', 'duration_ms'])
# {'duration_ms': 280000, 'name': 'You Can Call Me Al', 'popularity': 74}

{'name': 'You Can Call Me Al', 'popularity': 76, 'duration_ms': 280000}

Now write a method called `extract_tracks_info` that takes in an argument of a list of tracks, and has a default argument of `keys = ['name', 'id', 'popularity']` and retrieves the corresponding data for a list of tracks. 

In [22]:
def extract_tracks_info(tracks_response, keys = ['name', 'id', 'popularity']):
    return [extract_track_info(track) for track in tracks_response]

In [23]:
top_songs_info = extract_tracks_info(simon_tracks['tracks'])

top_songs_info[:2]
# [{'song': 'You Can Call Me Al',
#   'id': '0qxYx4F3vm1AOnfux6dDxP',
#   'popularity': 74},
#  {'song': 'Me and Julio Down by the Schoolyard',
#   'id': '6vxHp3CDNo0afgKGp2yi1E',
#   'popularity': 70},

[{'name': 'You Can Call Me Al',
  'id': '0qxYx4F3vm1AOnfux6dDxP',
  'popularity': 76},
 {'name': 'Me and Julio Down by the Schoolyard',
  'id': '6vxHp3CDNo0afgKGp2yi1E',
  'popularity': 72}]

## Song Analysis 

Now write a function that takes in a `track_id` and returns the url to retrieve the features of a track.

In [24]:
def audio_features(track_id):
    return f'https://api.spotify.com/v1/audio-features/{track_id}'

In [25]:
top_songs_info[0]['id']

'0qxYx4F3vm1AOnfux6dDxP'

In [26]:
top_song_id = top_songs_info[0]['id']
audio_url = audio_features(top_song_id)

audio_url
# 'https://api.spotify.com/v1/audio-features/0qxYx4F3vm1AOnfux6dDxP'

'https://api.spotify.com/v1/audio-features/0qxYx4F3vm1AOnfux6dDxP'

Now use this url to retrieve the characteristics of this song.

In [27]:
audio_data = make_request(audio_url)

In [28]:
audio_data
# {'danceability': 0.776,
#  'energy': 0.763,
#  'key': 5,
#  'loudness': -8.124,
#  'mode': 1,
#  'speechiness': 0.0535,
#  'acousticness': 0.182,
#  'instrumentalness': 0.0065,
#  'liveness': 0.077,
#  'valence': 0.82,
#  'tempo': 128.433,
#  'type': 'audio_features',
#  'id': '0qxYx4F3vm1AOnfux6dDxP',
#  'uri': 'spotify:track:0qxYx4F3vm1AOnfux6dDxP',
#  'track_href': 'https://api.spotify.com/v1/tracks/0qxYx4F3vm1AOnfux6dDxP',
#  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0qxYx4F3vm1AOnfux6dDxP',
#  'duration_ms': 280000,
#  'time_signature': 4}

{'danceability': 0.776,
 'energy': 0.763,
 'key': 5,
 'loudness': -8.124,
 'mode': 1,
 'speechiness': 0.0534,
 'acousticness': 0.182,
 'instrumentalness': 0.00647,
 'liveness': 0.077,
 'valence': 0.82,
 'tempo': 128.433,
 'type': 'audio_features',
 'id': '0qxYx4F3vm1AOnfux6dDxP',
 'uri': 'spotify:track:0qxYx4F3vm1AOnfux6dDxP',
 'track_href': 'https://api.spotify.com/v1/tracks/0qxYx4F3vm1AOnfux6dDxP',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0qxYx4F3vm1AOnfux6dDxP',
 'duration_ms': 280000,
 'time_signature': 4}

Next write a function called `extract_audio_data` that takes in a track features response, and only returns the following selected attributes by default:

`['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence']`

But can also return a list of other attributes if provided.

In [29]:
def extract_audio_data(audio_data, selected_attrs=['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
                                                   'instrumentalness', 'liveness', 'valence']):
    return {k: v for k,v in audio_data.items() if k in selected_attrs}

In [30]:
extract_audio_data(audio_data)
# {'danceability': 0.776,
#  'energy': 0.763,
#  'loudness': -8.124,
#  'speechiness': 0.0535,
#  'acousticness': 0.182,
#  'instrumentalness': 0.0065,
#  'liveness': 0.077,
#  'valence': 0.82}

{'danceability': 0.776,
 'energy': 0.763,
 'loudness': -8.124,
 'speechiness': 0.0534,
 'acousticness': 0.182,
 'instrumentalness': 0.00647,
 'liveness': 0.077,
 'valence': 0.82}

In [31]:
alt_attrs = ['danceability', 'energy', 'loudness', 'instrumentalness']
extract_audio_data(audio_data, selected_attrs = alt_attrs)
# {'danceability': 0.776,
#  'energy': 0.763,
#  'loudness': -8.124,
#  'instrumentalness': 0.0065}

{'danceability': 0.776,
 'energy': 0.763,
 'loudness': -8.124,
 'instrumentalness': 0.00647}

Now let's work on constructing a request to retreive the song features for a list of songs.  First, let's get a list of the song ids for our top songs.

In [32]:
top_song_ids = [top_song['id'] for top_song in top_songs_info]
# ['0qxYx4F3vm1AOnfux6dDxP',
#  '6vxHp3CDNo0afgKGp2yi1E',
#  '6Qb7gtV6Q4MnUjSbkFcopl',
#  '51KKQAgYFoJHgVIuJWHdHb',
#  '71GvlH0VdeClloLIkHrAVu',
#  '3f0U5NaD1bCk8nmKpn2ZJY',
#  '3gIBSlXYIN1mru35l4LWPB',
#  '2h23bjG8B3bcD47HBu6bHG',
#  '0zb2kpEQMnqJPiLACKMiFM',
#  '00IrSynHsun7DpDrLkRIjM']

From there let's write a function that takes in a list of track ids and constructs the url to retreive each of the provided songs' features -- we can see how to do that [here](https://developer.spotify.com/documentation/web-api/reference/tracks/get-several-audio-features/).

In [33]:
def audio_data_url(track_ids):
    return 'https://api.spotify.com/v1/audio-features/?ids=' + ','.join(track_ids)

In [34]:
songs_url = audio_data_url(top_song_ids)
songs_url
# 'https://api.spotify.com/v1/audio-features/?ids=
# 0qxYx4F3vm1AOnfux6dDxP,6vxHp3CDNo0afgKGp2yi1E,
# 6Qb7gtV6Q4MnUjSbkFcopl,51KKQAgYFoJHgVIuJWHdHb,71GvlH0VdeClloLIkHrAVu,
# 3f0U5NaD1bCk8nmKpn2ZJY,3gIBSlXYIN1mru35l4LWPB,2h23bjG8B3bcD47HBu6bHG,
# 0zb2kpEQMnqJPiLACKMiFM,00IrSynHsun7DpDrLkRIjM'

'https://api.spotify.com/v1/audio-features/?ids=0qxYx4F3vm1AOnfux6dDxP,6vxHp3CDNo0afgKGp2yi1E,6Qb7gtV6Q4MnUjSbkFcopl,3f0U5NaD1bCk8nmKpn2ZJY,51KKQAgYFoJHgVIuJWHdHb,71GvlH0VdeClloLIkHrAVu,2h23bjG8B3bcD47HBu6bHG,3gIBSlXYIN1mru35l4LWPB,0zb2kpEQMnqJPiLACKMiFM,5vZ1BKMSLgrxxPYGMR904n'

From there, let's request the song information.

In [35]:
songs_features = make_request(songs_url)
# songs_features

# [{'danceability': 0.776,
#   'energy': 0.763,
#   'key': 5,
#   'loudness': -8.124,
#   'mode': 1,
#   'speechiness': 0.0535,
#   'acousticness': 0.182,
#   'instrumentalness': 0.0065,
#   'liveness': 0.077,
#   'valence': 0.82,
#   'tempo': 128.433,
#   'type': 'audio_features',
#   'id': '0qxYx4F3vm1AOnfux6dDxP',
#   'uri': 'spotify:track:0qxYx4F3vm1AOnfux6dDxP',
#   'track_href': 'https://api.spotify.com/v1/tracks/0qxYx4F3vm1AOnfux6dDxP',
#   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0qxYx4F3vm1AOnfux6dDxP',
#   'duration_ms': 280000,
#   'time_signature': 4},
#  {'danceability': 0.626,
#   'energy': 0.816,
#   'key': 9,
#   'loudness': -10.593,
#   'mode': 1,

In [36]:
songs_features

{'audio_features': [{'danceability': 0.776,
   'energy': 0.763,
   'key': 5,
   'loudness': -8.124,
   'mode': 1,
   'speechiness': 0.0534,
   'acousticness': 0.182,
   'instrumentalness': 0.00647,
   'liveness': 0.077,
   'valence': 0.82,
   'tempo': 128.433,
   'type': 'audio_features',
   'id': '0qxYx4F3vm1AOnfux6dDxP',
   'uri': 'spotify:track:0qxYx4F3vm1AOnfux6dDxP',
   'track_href': 'https://api.spotify.com/v1/tracks/0qxYx4F3vm1AOnfux6dDxP',
   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0qxYx4F3vm1AOnfux6dDxP',
   'duration_ms': 280000,
   'time_signature': 4},
  {'danceability': 0.626,
   'energy': 0.816,
   'key': 9,
   'loudness': -10.593,
   'mode': 1,
   'speechiness': 0.131,
   'acousticness': 0.237,
   'instrumentalness': 1.8e-06,
   'liveness': 0.104,
   'valence': 0.798,
   'tempo': 105.307,
   'type': 'audio_features',
   'id': '6vxHp3CDNo0afgKGp2yi1E',
   'uri': 'spotify:track:6vxHp3CDNo0afgKGp2yi1E',
   'track_href': 'https://api.spotify.com/v1/tracks/

From there, we can use our extract_audio_data function to limit the keys of limit the data for each song.

In [37]:
[extract_audio_data(audio_data) for audio_data in songs_features['audio_features']][:3]

[{'danceability': 0.776,
  'energy': 0.763,
  'loudness': -8.124,
  'speechiness': 0.0534,
  'acousticness': 0.182,
  'instrumentalness': 0.00647,
  'liveness': 0.077,
  'valence': 0.82},
 {'danceability': 0.626,
  'energy': 0.816,
  'loudness': -10.593,
  'speechiness': 0.131,
  'acousticness': 0.237,
  'instrumentalness': 1.8e-06,
  'liveness': 0.104,
  'valence': 0.798},
 {'danceability': 0.815,
  'energy': 0.372,
  'loudness': -12.814,
  'speechiness': 0.0752,
  'acousticness': 0.166,
  'instrumentalness': 0.000116,
  'liveness': 0.0767,
  'valence': 0.293}]

### Resources

[StackOverflow Post](https://stackoverflow.com/questions/30557409/spotify-api-post-call-response-415)

[Spotify Tutorial Python](https://kholinlabs.com/how-oauth-works-with-spotify-as-an-example)